In [ ]:
!pip install ../input/pretrainedmodels/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4/ > /dev/null
package_path = '../input/unetmodelscript'
import sys
sys.path.append(package_path)


In [ ]:
cd '../input/attunetpytorch/EfficientUnet_pytorch'

In [ ]:
from efficientunet import *

In [ ]:
cd "../../../working"

In [ ]:
cd "../input/attentionsmp"

In [ ]:
 import segmentation_models_pytorch as smp

In [ ]:
cd "../../working"

In [ ]:
import pdb
import os
import cv2
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset

from torchvision.transforms import ToTensor,Compose,Normalize
from torchvision.transforms import ToPILImage
import torchvision

import torch.utils.data as data

import torchvision.transforms
from PIL import Image

In [ ]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
class TestDataset(Dataset):
    '''Dataset for test prediction'''
    def __init__(self, root, df, mean, std,i):
        self.root = root
        df['ImageId'] = df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
        self.fnames = df['ImageId'].unique().tolist()
        self.num_samples = len(self.fnames)
        if(i==1):
            
            
            self.transform = Compose([ToTensor(),Normalize(mean=mean, std=std)])
                
        if(i==2):
            self.transform = Compose([HorizontalFlip(p=1),Normalize(mean=mean, std=std, p=1),ToTensor()])

        if(i==3):
             self.transform = Compose([VerticalFlip(p=1),Normalize(mean=mean, std=std, p=1),ToTensor()])

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = os.path.join(self.root, fname)
        
        image = cv2.imread(path)
        images = self.transform(image)
        return fname, images

    def __len__(self):
        return self.num_samples

In [ ]:
def post_process(probability,threshold , min_size):
    '''Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored'''
    
    num_component, component = cv2.connectedComponents(probability.astype(np.uint8))
    
    predictions = np.zeros((256, 1600), np.float32)

    num = 0
    for c in range(1, num_component):
        p = (component == c)
        
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions

In [ ]:
sample_submission_path = '../input/severstal-steel-defect-detection/sample_submission.csv'
test_data_folder = "../input/severstal-steel-defect-detection/test_images"

In [ ]:
best_threshold = 0.5
num_workers = 2
batch_size = 1
print('best_threshold', best_threshold)
min_size = 3500
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
df = pd.read_csv(sample_submission_path)

testset = DataLoader(
    TestDataset(test_data_folder, df, mean, std,1),
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True)


In [ ]:
!ls "../input/atteffunetcombo"

In [ ]:
ckpt_path = "../input/atteffunetcombo/atteffunetcomboloss1.pth"
device = torch.device("cuda")
model = get_efficientunet_b4(out_channels=4, concat_input=True, pretrained=False)
model.to(device)
model.eval()
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model.load_state_dict(state["state_dict"])

In [ ]:
ckpt_path = "../input/attefbceloss4/atteffunetbceloss2.pth"
device = torch.device("cuda")
model2 = get_efficientunet_b4(out_channels=4, concat_input=True, pretrained=False)
model2.to(device)
model2.eval()
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model2.load_state_dict(state["state_dict"])

In [ ]:
ckpt_path = "../input/attres182/attres18unetbceloss3.pth"
device = torch.device("cuda")
model3 = smp.Unet("resnet18", encoder_weights=None, classes=4, activation=None,attention_type="scse")
model3.to(device)
model3.eval()
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model3.load_state_dict(state["state_dict"])

In [ ]:
ckpt_path = "../input/unetstartermodelfile4/model89088.pth"
device = torch.device("cuda")
model4 = smp.Unet("resnet18", encoder_weights=None, classes=4, activation=None)
model4.to(device)
model4.eval()
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model4.load_state_dict(state["state_dict"])

In [ ]:
predictions = []
for i, batch in enumerate(tqdm(testset)):
    fnames, images = batch
    
    py = (torch.sigmoid(model(images.to(device)))+ (torch.sigmoid(model(images.flip(3).to(device)))).flip(3))/2
    batch_preds = 0.2518*py
    
    py = (torch.sigmoid(model2(images.to(device)))+ (torch.sigmoid(model2(images.flip(3).to(device)))).flip(3))/2
    batch_preds += 0.252*py
    
    py = (torch.sigmoid(model3(images.to(device))) + (torch.sigmoid(model3(images.flip(3).to(device)))).flip(3))/2
    batch_preds += 0.249*py
    
    py = (torch.sigmoid(model4(images.to(device))) + (torch.sigmoid(model4(images.flip(3).to(device)))).flip(3))/2
    batch_preds += 0.2472*py
    
    batch_preds = batch_preds.detach().cpu().numpy()
    
    j=0
    for fname, preds in zip(fnames, batch_preds):
        
        for cls, pred in enumerate(preds):    
            #predict=pred2[cls]
            min_size=2000
            th=0.55
            if cls==0:
                min_size=700    
            if cls==2:
                th=0.65
            if cls == 1:
                th=0.85
            pred=(pred>th).astype(int)
            

            pred = post_process(pred, 0.45, min_size)
            
            if pred.sum()>0:
                print(cls,pred.sum())
                
            rle = mask2rle(pred)
            name = fname + f"_{cls+1}"
            predictions.append([name, rle])


 
               
df = pd.DataFrame(predictions, columns=['ImageId_ClassId', 'EncodedPixels'])
df.to_csv("submission.csv", index=False)    

In [ ]:
df.head()